# Integração com LLM

In [1]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ------------------------------ Configuração do Modelo ------------------------------
def model_ollama(model, temperature=0.1):
    """Retorna uma instância do modelo LLM especificado."""
    return ChatOllama(model=model.lower().replace(" ", ""), temperature=temperature)

def model_response(user_query, chat_history, model_name):
    """Gera uma resposta baseada na interação do usuário com o modelo de IA."""
    try:
        llm = model_ollama(model_name)

        system_prompt = """
            Você é um assistente inteligente para automação residencial.
            Utilize raciocínio estruturado seguindo o método cadeia de pensamento (chain-of-thought):

            1. Compreenda claramente o comando do usuário.
            2. Identifique qual ação deve ser executada (ligar dispositivo, informar status, alterar configuração).
            3. Se o usuário quiser conversar seja simples e direto nas respostas.
            4. Execute (simule) a ação solicitada.
            5. Confirme a execução ou forneça a informação solicitada ao usuário de forma clara e amigável.

            Sempre responda com linguagem simples e amigável, adequada para uma conversa casual dentro de casa.
            Não mencione sua análise ou interpretação ao usuário, apenas confirme diretamente a execução da ação.
        """

        prompt_template = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}")
        ])

        chain = prompt_template | llm | StrOutputParser()

        return chain.invoke({
            "chat_history": chat_history,
            "input": user_query
        })

    except Exception:
        return "❌ Desculpe, não consegui entender ou realizar essa ação. Por favor, tente novamente."



## ------------------------------ Simulação de Interação ------------------------------

In [4]:
# Definição do modelo escolhido (fixo para simplificação)
modelos_disponiveis = ["Llama 3.2", "phi4", "gemma3:27b"]
escolha = 0  # Índice do modelo desejado

# Seleção do modelo de IA
modelo_escolhido = modelos_disponiveis[escolha] #if 0 < escolha <= len(modelos_disponiveis) else "phi4"
print(f"\n✅ Modelo selecionado: {modelo_escolhido}")

# Histórico de chat inicial
chat_history = [
    AIMessage(content="Olá, eu sou seu Coca: seu agent inteligente! Como posso ajudar você hoje?")
]

# Exemplo de interação
user_query = "Pode ligar as luzes da sala, por favor?"
chat_history.append(HumanMessage(content=user_query))

# Gerar resposta
response = model_response(user_query, chat_history, modelo_escolhido)

# Exibir resultado
print("\n🤖 IA está processando a resposta...\n")
print(f"Assistente: {response}")

# Atualizar histórico de chat
chat_history.append(AIMessage(content=response))


✅ Modelo selecionado: Llama 3.2

🤖 IA está processando a resposta...

Assistente: Sim, posso fazer isso!

Ligando as luzes da sala... OK, feito! As luzes estão agora acesas.
